In [1]:
import numpy as np
import pandas as pd
import urllib
import xml.etree.ElementTree as ET

In [2]:
data = pd.read_csv("../../gci_project_storage/mynavi_data.csv", encoding="SHIFT-JIS")

In [3]:
data.shape

(46469, 110)

In [ ]:
serviceurl = 'http://maps.googleapis.com/maps/api/geocode/xml?'

print(inputting "" terminates this program)

while Ture:
    address = input('Enter location: ')
    if len(address) < 1 : break

    url = serviceurl + urllib.parse.urlencode({'sensor':'false', 'address': address})
    print ('Retrieving', url)
    uh = urllib.request.urlopen(url)
    data = uh.read()
    print ('Retrieved',len(data),'characters')
    print (data)
    print ()
    tree = ET.fromstring(data)


    results = tree.findall('result')
    lat = results[0].find('geometry').find('location').find('lat').text
    lng = results[0].find('geometry').find('location').find('lng').text
    location = results[0].find('formatted_address').text

    print ('lat',lat,'lng',lng)
    print (location)
    